<h3 align="center">Segmenting and Clustering Neighborhoods in Toronto</h3>
<h5 align=center>Coursera Course 9 Week-3 Assignment <b>Part-B</b> Sagar Shah</h5>

(1) Using Beautifulsoup package to scrap html from wikipedia page  
(2) Using pandas package to create dataframe


#### Installing and importing necessary packages

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
beautifulsoup4            4.6.3                    py35_0    conda-forge


#### Get the Wikipedia package for Canada postal code

In [3]:
htmldoc = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

#### Instantiate Beautiful Soup

In [4]:
soupobj = BeautifulSoup(htmldoc,"lxml")

#### Scrap the table for postal code from html page

In [5]:
toronto_table = soupobj.find_all('table',class_ = 'wikitable sortable')
toronto_table = toronto_table[0]
records = toronto_table.findAll("tr")


#### Output the table to CSV file

In [6]:
csvfile = open('Toronto_postcode.csv','wt',newline='')
writer = csv.writer(csvfile)


In [7]:
try:
   for row in records:
       csvrow = []
       for cell in row.findAll(['td','th']):
           csvrow.append(cell.get_text())
       writer.writerow(csvrow)
finally:
   csvfile.close()

#### Read CSV and create pandas Dataframe

In [8]:
df = pd.read_csv('Toronto_postcode.csv') 
df.shape

(289, 3)

In [9]:
df=df.rename(columns = {'Neighbourhood\n':'Neighbourhood'})
df['Neighbourhood']= df['Neighbourhood'].str.replace(r'\n', '')
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.shape

(212, 3)

### Group all the neighbourhoods in same Borough

In [10]:
df = df.groupby(['Borough','Postcode'], as_index=False, sort=False).agg(', '.join)


### Update Neighbourhood with same value from Borough when it is Not Assigned

In [11]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'] , df['Neighbourhood'])
df.head(1)

,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods


Combining 2 columns to form Address field, however it is not needed anymore since geocode API which needs it; does not work anymore

In [12]:
df["Address"]= df['Borough'] + ", " + df['Postcode'] 
df.head(1)

,Borough,Postcode,Neighbourhood,Address
0,North York,M3A,Parkwoods,"North York, M3A"


Following part is not needed anymore since I kept on getting Over query Limit error

In [13]:
#!conda install -c conda-forge geocoder --yes
#import geocoder

In [14]:
#g = geocoder.google('Mountain View, CA')
#g.latlong
#g

#### Getting Postal code to coodinates file into Dataframe 

In [15]:
!wget -q -O 'Geospatial_Coordinates.csv'  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
print('Data downloaded!')

Data downloaded!


In [17]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')

#### Joining two Dataframes on postal code to get coordinates

In [18]:
final_df = pd.merge(df, geo_df, left_on='Postcode', right_on='Postal Code')

In [20]:
final_df = final_df[['Borough','Postcode','Neighbourhood','Address','Latitude','Longitude']]
final_df.head(12)

,Borough,Postcode,Neighbourhood,Address,Latitude,Longitude
0,North York,M3A,Parkwoods,"North York, M3A",43.753259,-79.329656
1,North York,M4A,Victoria Village,"North York, M4A",43.725882,-79.315572
2,Downtown Toronto,M5A,"Harbourfront, Regent Park","Downtown Toronto, M5A",43.654260,-79.360636
3,North York,M6A,"Lawrence Heights, Lawrence Manor","North York, M6A",43.718518,-79.464763
4,Queen's Park,M7A,Queen's Park,"Queen's Park, M7A",43.662301,-79.389494
5,Etobicoke,M9A,Islington Avenue,"Etobicoke, M9A",43.667856,-79.532242
6,Scarborough,M1B,"Rouge, Malvern","Scarborough, M1B",43.806686,-79.194353
7,North York,M3B,Don Mills North,"North York, M3B",43.745906,-79.352188
8,East York,M4B,"Woodbine Gardens, Parkview Hill","East York, M4B",43.706397,-79.309937
9,Downtown Toronto,M5B,"Ryerson, Garden District","Downtown Toronto, M5B",43.657162,-79.378937
